# Gradient boosting

## Load libraries

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import warnings 
warnings.filterwarnings("ignore")
sns.set(style="white", color_codes=True)
sns.set_context(rc={"font.family":'sans',"font.size":10,"axes.titlesize":24,"axes.labelsize":24}) 
from sklearn import tree
from sklearn import linear_model
from sklearn import neighbors
from scipy import stats
from sklearn.metrics import mean_squared_error,r2_score

## Load data

In [2]:
X_train_complete = pd.read_csv("TrainClean.csv")
X_test = pd.read_csv("TestClean.csv")
#X_PCA = pd.read_csv("TrainClean10PCA.csv")
#X_train_complete.describe()

In [3]:
y=X_train_complete['NumberOfSales'].values.reshape(-1,1)
x=X_train_complete.drop(columns=[X_train_complete.columns[0],'NumberOfSales'],axis=1)

## Gradient Boosting

Step 1 generating the first model

In [20]:
perf=[]
model_list=[] # lol
prediction = np.zeros((len(y),1))
clf = tree.DecisionTreeRegressor(max_depth=1)
#clf = linear_model.LinearRegression()
#clf=neighbors.KNeighborsRegressor(2)
clf = clf.fit(x, y)
model_list.append(clf)
yp = clf.predict(x).reshape(-1,1)
prediction += yp
y_residual = y-yp
perf.append((mean_squared_error(y,prediction),r2_score(y,prediction)))

Step 2 loop generating models to describe residuals of the previous models

In [21]:
no_boosting_runs = 100
for i in range(no_boosting_runs):
    clf = tree.DecisionTreeRegressor(max_depth=1)
    #clf = linear_model.LinearRegression()
    # suggested no_boosting_runs = 3 for KNN
    #clf=neighbors.KNeighborsRegressor(2)
    clf = clf.fit(x, y_residual)
    model_list.append(clf)
    yp = clf.predict(x).reshape(-1,1)
    # update the prediction
    prediction += yp
    y_residual = y-prediction
    perf.append((mean_squared_error(y,prediction),r2_score(y,prediction)))

In [22]:
print("RMSE , R2")
perf[-20:]

RMSE , R2


[(0.0018000426721939485, 0.28923731601595326),
 (0.0017991798625533835, 0.28957800399261546),
 (0.0017984929344419564, 0.2898492436002782),
 (0.0017976338928460981, 0.29018844373134056),
 (0.0017967801196504918, 0.2905255635882177),
 (0.001796020411167591, 0.2908255411658014),
 (0.0017953474026638871, 0.29109128449390465),
 (0.0017946860958872168, 0.2913524072921494),
 (0.0017940527154780817, 0.2916025031185061),
 (0.001793400830406458, 0.29185990567363096),
 (0.0017925071267791625, 0.29221279241272613),
 (0.0017918134543887588, 0.29248669505827174),
 (0.0017911295371122393, 0.2927567458112851),
 (0.0017902656765987816, 0.2930978487343533),
 (0.0017890385805468538, 0.293582378405148),
 (0.0017879655476953577, 0.2940060748659674),
 (0.001786834326092614, 0.29445274766709983),
 (0.0017857542916084191, 0.2948792087841833),
 (0.0017849303779087812, 0.29520453835640503),
 (0.0017842526275646908, 0.29547215404189864)]

Define a function to perform model evaluation

In [23]:
def GradientBoosting_eval(models,x):
    prediction = np.zeros((len(x),1))
    for model in models:
        yp = model.predict(x).reshape(-1,1)
        prediction += yp
    return prediction.reshape(-1,1)

result=GradientBoosting_eval(model_list,x)

In [24]:
result

array([[2.26043626],
       [2.26043626],
       [2.26043626],
       ...,
       [2.1862779 ],
       [2.19320022],
       [2.19715671]])

In [25]:
model_list

[DecisionTreeRegressor(criterion='mse', max_depth=1, max_features=None,
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best'),
 DecisionTreeRegressor(criterion='mse', max_depth=1, max_features=None,
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best'),
 DecisionTreeRegressor(criterion='mse', max_depth=1, max_features=None,
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best'),
 DecisionTreeRegressor(criterion='mse', max_depth=1,